
# About RI_HEVNAA

It is recomended you use Python 3.10 >

RI_HEVNAA is open source, with all code available for download, inspection and modification at [https://github.com/UMN-VR/RI_HEVNAA](https://github.com/UMN-VR/RI_HEVNAA)

# LLM Feedback

## OpenAI

### Setup

#### Load creds.env file

The file 'creds.env' needs to have a line 'OPENAI_API_KEY=...'

    True = Loaded .env file, False = fail


In [2]:
import dotenv

dotenv.load_dotenv('creds.env')

True

#### Install pip dependencies 

Should look like this

    Requirement already satisfied: openai in /usr/local/lib/python3.10/site-packages (0.28.0)
    Requirement already satisfied: tiktoken in /usr/local/lib/python3.10/site-packages (0.4.0)

    
    ...
    

    Requirement already satisfied: exceptiongroup>=1.0.2 in /usr/local/lib/python3.10/site-packages (from anyio<5,>=3.4.0->starlette<0.28.0,>=0.27.0->fastapi<0.100.0,>=0.95.2->chromadb) (1.1.3)


In [3]:
!pip install openai tiktoken chromadb langchain

### Run

#### Import Statements

In [4]:

import dotenv
import os
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

#### Load repo_path

In [5]:
working_dir = os.getcwd()

repo_path = working_dir

print(f"repo_path: {repo_path}")

repo_path: /Library/Github/RI-HEVNAA


In [6]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/**/*",
    suffixes=[".py", ".ipynb", ".md", ".log"],
    parser=LanguageParser()
)

#add 
documents = loader.load()

print(f"Found {len(documents)} documents.")

Found 37 documents.


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
markdown_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN,
                                                                    chunk_size=2000,
                                                                    chunk_overlap=200)
texts = python_splitter.split_documents(documents)
texts += markdown_splitter.split_documents(documents)
len(texts)

230

In [8]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
llm = ChatOpenAI(model_name="gpt-4") 
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

#### Create Conversation

In [10]:
messages = []
question = None
answer = None

#### Print Messages

In [11]:
print(f"{messages}")

[]


#### Update Conversation

In [80]:
if question is None:
    print("Ask a question first!") 
else:
    messages.append(HumanMessage(content=question))
    messages.append(AIMessage(content=answer))
    print(messages)

[HumanMessage(content='Tell me about RI-HEVNAA', additional_kwargs={}, example=False), AIMessage(content="RI-HEVNAA, which stands for Real-time Interrupt-driven Hemispheric Emulated von Neumann Agent Architecture, is an advanced chatbot architecture designed to replicate the structure and functionality of the human brain. The architecture draws inspiration from principles of biomimicry and von Neumann's computational model, and it provides a comprehensive framework for building sophisticated and human-like chatbot systems.\n\nThe RI-HEVNAA architecture comprises several key components. These include SuperUnits, each equipped with an Agent Central Processing Unit (A-CPU) and specialized Accelerators. SuperUnits include the Agent SuperUnit (A_SU), Left Hemisphere SuperUnit (LH_SU), and Right Hemisphere SuperUnit (RH_SU), each dedicated to different tasks similar to the human brain's division of labor.\n\nThe architecture also features real-time interaction with hardware components, enabl

In [ ]:
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

combine_docs_chain = StuffDocumentsChain()
vectorstore = 
retriever = vectorstore.as_retriever()

# This controls how the standalone question is generated.
# Should take `chat_history` and `question` as input variables.
template = (
    "Combine the chat history and follow up question into "
    "a standalone question. Chat History: {chat_history}"
    "Follow up question: {question}"
)
prompt = PromptTemplate.from_template(template)
llm = OpenAI()
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
chain = ConversationalRetrievalChain(
    combine_docs_chain=combine_docs_chain,
    retriever=retriever,
    question_generator=question_generator_chain,
)

#### Chat

In [112]:
llm(messages)

AIMessage(content='I\'m sorry, but I couldn\'t find any information on "RI-HEVNAA". It\'s possible that there may be a typo or misunderstanding in the term. Please provide more context or check the spelling so I can give you an accurate response.', additional_kwargs={}, example=False)

Test:


In [115]:
memory.chat_memory = messages

In [116]:
# print & render newline characters
print(memory.chat_memory)

[HumanMessage(content='Tell me about RI-HEVNAA', additional_kwargs={}, example=False), AIMessage(content="RI-HEVNAA, which stands for Real-time Interrupt-driven Hemispheric Emulated von Neumann Agent Architecture, is an advanced chatbot architecture designed to replicate the structure and functionality of the human brain. The architecture draws inspiration from principles of biomimicry and von Neumann's computational model, and it provides a comprehensive framework for building sophisticated and human-like chatbot systems.\n\nThe RI-HEVNAA architecture comprises several key components. These include SuperUnits, each equipped with an Agent Central Processing Unit (A-CPU) and specialized Accelerators. SuperUnits include the Agent SuperUnit (A_SU), Left Hemisphere SuperUnit (LH_SU), and Right Hemisphere SuperUnit (RH_SU), each dedicated to different tasks similar to the human brain's division of labor.\n\nThe architecture also features real-time interaction with hardware components, enabl

In [101]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [1]:
question = input("Ask a question: ")
print(f"Question:\n{question}")
result = qa(question)
answer = result['answer']
print(answer)


Question:
Tell me about RI_HEVNAA


NameError: name 'qa' is not defined

# Local Execution

In [1]:
import os
!python3 main.py

@main: Starting RI-HEVNAA System  with dir: /Library/Github/RI-HEVNAA
Deleting logs

@RIHEVNAA.init:

@BusManager.init:
@WebMonitoringProcess.init:
@Accelerators.init:
@RIHEVNAA.init: DONE: Init Components
Initializing Interhemispheric Units
@Unit.init: Starting A_HCPU
@A_HCPU.link_accelerators:
@A_HCPU.init:
Loaded sub-pathway: pathways/hello_world/pathway.json
Init Control Unit DONE
Init Execution Unit DONE
Init Memory Unit DONE
@Unit.init: Starting R_HCPU
@R_HCPU.link_accelerators:
@R_HCPU.init:
Loaded sub-pathway: pathways/hello_world/pathway.json
Init Control Unit DONE
Init Execution Unit DONE
Init Memory Unit DONE
@Unit.init: Starting L_HCPU
@L_HCPU.link_accelerators:
@L_HCPU.init:
Loaded sub-pathway: pathways/hello_world/pathway.json
Init Control Unit DONE
Init Execution Unit DONE
Init Memory Unit DONE

@RIHEVNAA.execute: Starting RI-HEVNAA Execution

Creating process for bus_manager
Creating process for web_monitoring_process
Creating process for accelerators
Creating process f

**Troubleshooting 1:** 


**Troubleshooting 2:**
See further advice in the FAQ if you run into trouble:
https://github.com/UMN-VR/RI-HEVNAA/blob/master/docs/FAQ.md



# Setup and run RI_HEVNAA with Google Colab

As many users do not have a powerful enough CPU or GPU installed on their computer, this guide will explain how to use RI_HEVNAA with a free CPU & GPU from google via their google colab service


## Optional: [Read papers](https://github.com/UMN-VR/RI-HEVNAA/tree/main/papers)

This will help you understand the related concepts, purpose and capabilities of RI_HEVNAA.

[Code Llama: Open Foundation Models for Code](https://github.com/UMN-VR/RI-HEVNAA/blob/main/papers/Code%20Llama%20Open%20Foundation%20Models%20for%20Code.pdf)

[Consciousness in Artificial Intelligence Insights from the Science of Consciousness](https://github.com/UMN-VR/RI-HEVNAA/blob/main/papers/Consciousness%20in%20Artificial%20Intelligence%20Insights%20from%20the%20Science%20of%20Consciousness.pdf)

**Note:** This web page is a type of web application called a jupyter notebook. Jupyter notebooks are open source web applications that allow you to write and run live code.

**Note:** I have written either 'Do every time' or 'First time only' to indicate if an instruction is something you only need to do once when first setting up the system or each time you want to run the RI_HEVNAA server.


**Note:** Colab can be a bit slow.

## Change runtime type. (First time only)

1. Select 'Runtime' in the top menu (of this notebook).
2. Select 'Change runtime
type'
3. Select CPU 
4. Click SAVE

## Mount drive so data can be shared with client. (Do every time)

To run the following block of code. Hover your mouse over it and then click the play icon on the left of it.

**Note**: This may take you to another page where you need to authenticate with your google account.

If you find it takes ages (over two minutes) without doing anything then click the stop icon and then play again.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')



## Clone RI_HEVNAA repo (First time only, or if you need to update to the latest version)






This is done in order to download the source code into google drive so we can run the server application.


In [ ]:
import os
assert os.path.isdir('/content/drive/MyDrive'), 'Are you sure the drive is mounted? See Mount drive step above'
!rm -r  /content/drive/'MyDrive'/RI_HEVNAA
!pip install imagecodecs # this is required to enable support for certain types of compressed tif images.
!git clone https://github.com/UMN-VR/RI_HEVNAA.git /content/drive/'MyDrive'/RI_HEVNAA

## Prepare working directory



In [ ]:
import os
assert os.path.isdir('/content/drive/MyDrive/RI_HEVNAA'), 'Are you sure the drive is mounted? See Mount drive step above'
!mkdir /content/drive/MyDrive/'RI_HEVNAA'/logs


## Start server (Do every time)

**Note** This cell should keep running until you stop it. Which you can do after you have finished using RI_HEVNAA.

**Important** If you are using this Python Notebook on your local machine, run the code in [Local Execution](#local-execution)


### Execute RI_HEVNAA on Google Colab

In [ ]:
import os
assert os.path.isdir('/content/drive/MyDrive/RI_HEVNAA'), 'Are you sure the drive is mounted? See Mount drive step above'
!python /content/drive/MyDrive/RI_HEVNAA/main.py --dir '/content/drive/MyDrive/RI_HEVNAA' 

**Troubleshooting 1:** If running the code above gives the output "can't open file [..] main.py" then you probably need to mount google drive. Go back to the instruction _Mount drive so data can be shared with client. (Do every time)_

**Troubleshooting 2:** If starting the server stopped working for you then you may need to run the 'Clone RI_HEVNAA repository' above as the code now receives the syncdir as a command line argument but you will need to update to the latest version (by running the clone command) for this to work.

**Troubleshooting 3:** If it appears the client is _very_ slow it may be that other files are still syncing. On mac and windows you can inspect the google drive sync activity. It may be that client server interaction is interrupted by large amounts of sync activity from newly added datasets or other files you have in google drive.

**Troubleshooting 4:**
See further advice in the FAQ if you run into trouble:
https://github.com/UMN-VR/RI-HEVNAA/blob/master/docs/FAQ.md



### Limitations of Google Colab

In Colab, you get around 12 hours of execution time but the session will disconnect if you are idle for 1 hour, meaning that every 12 hours, Disk, Memory and CPU Cache will get erased.

For larger projects, I recommend using your own server or paperspace. Google cloud and amazon web services also have options for renting GPUs. It's also possible to get a paid colab account which apparently increases the reliability.



## Credits

[Abraham George Smith](https://www.researchgate.net/profile/Abraham-Smith-2) -author of Python notebook used as starting point for this interactive tutorial and main developer of the RootPainter software.

[R. Ford Denison](https://darwinianagriculture.wordpress.com/) - Testing,  guidance on usability issues and collaboration with AGS on both the introductory and explanatory text.